In [1]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install sklearn ... si no lo tienes, instálalo!
from collections import deque
import random
import numpy as np

# Longitud de la secuencia precedente para recolectar para la RNN
SEQ_LEN = 60

# Cuánto tiempo en el futuro estamos tratando de predecir
FUTURE_PERIOD_PREDICT = 8

# El ratio que queremos predecir
RATIO_TO_PREDICT = "BTC-USD"

# Función para clasificar si el valor futuro es mayor que el valor actual
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

def preprocess_df(df):
    df = df.drop(["future"], axis=1)  # Eliminamos la columna 'future' ya que no la necesitamos aquí

    for col in df.columns:  # Recorrer todas las columnas
        if col != "target":  # Normalizar todas las columnas excepto 'target'
            df[col] = df[col].pct_change()  # Cambio porcentual para normalizar diferentes criptomonedas
            df.dropna(inplace=True)  # Eliminar los valores NA creados por pct_change
            df[col] = preprocessing.scale(df[col].values)  # Escalar entre 0 y 1

    df.dropna(inplace=True)  # Limpiar nuevamente para asegurar que no queden NaNs
    sequential_data = []  # Lista que contendrá las secuencias
    prev_days = deque(maxlen=SEQ_LEN)  # Deque para mantener las secuencias con longitud máxima de SEQ_LEN

    for i in df.values:  # Iterar sobre los valores del DataFrame
        prev_days.append([n for n in i[:-1]])  # Almacenar todos menos el target
        if len(prev_days) == SEQ_LEN:  # Asegurarse de tener 60 secuencias
            sequential_data.append([np.array(prev_days), i[-1]])  # Añadir la secuencia y el target

    random.shuffle(sequential_data)  # Barajar los datos para asegurar aleatoriedad
    
    buys = []  # Lista para almacenar las secuencias de compra
    sells = []  # Lista para almacenar las secuencias de venta

    for seq, target in sequential_data:  # Iterar sobre los datos secuenciales
        if target == 0:  # Si es una "no compra"
            sells.append([seq, target])  # Añadir a la lista de ventas
        elif target == 1:  # Si el target es 1...
            buys.append([seq, target])  # Añadir a la lista de compras

    random.shuffle(buys)  # Barajar las compras
    random.shuffle(sells)  # Barajar las ventas

    lower = min(len(buys), len(sells))  # Obtener la longitud menor

    buys = buys[:lower]  # Asegurar que ambas listas tengan la misma longitud
    sells = sells[:lower]  # Asegurar que ambas listas tengan la misma longitud

    sequential_data = buys + sells  # Unir ambas listas
    random.shuffle(sequential_data)  # Otra vez barajar para mezclar las clases

    X = []
    y = []

    for seq, target in sequential_data:  # Iterar sobre los nuevos datos secuenciales
        X.append(seq)  # X son las secuencias
        y.append(target)  # y son los targets/etiquetas (compras vs ventas/no compras)

    return np.array(X), y  # Devolver X e y, y convertir X en un array de numpy

# Uso de una cadena sin procesar para evitar errores de escape de unicode
file_path = r"C:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\Modelo Para Predecir Criptomonedas\crypto_data\BTC-USD.csv"

# Leer el archivo inicial
df = pd.read_csv(file_path, names=['time', 'low', 'high', 'open', 'close', 'volume'])

# DataFrame principal vacío para almacenar todos los datos
main_df = pd.DataFrame()

# Las 4 criptomonedas que queremos considerar
ratios = ["BCH-USD", "BTC-USD", "ETH-USD", "LTC-USD"]

# Ruta base donde se encuentran los archivos de datos
base_path = r"C:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\Modelo Para Predecir Criptomonedas\crypto_data"

# Iterar sobre cada ratio
for ratio in ratios:
    print(ratio)
    
    # Obtener la ruta completa al archivo
    dataset = os.path.join(base_path, f"{ratio}.csv")
    
    # Leer el archivo específico
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    
    # Renombrar columnas de 'close' y 'volume' para incluir el ticker
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    # Establecer 'time' como índice para poder unirlos por este tiempo compartido
    df.set_index("time", inplace=True)
    
    # Ignorar otras columnas además de 'close' y 'volume'
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    # Si el DataFrame principal está vacío, simplemente asignar el DataFrame actual
    if len(main_df) == 0:
        main_df = df
    else:
        # De lo contrario, unir estos datos con el DataFrame principal
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # Si hay huecos en los datos, usar los valores conocidos previamente
main_df.dropna(inplace=True)  # Eliminar cualquier fila que aún tenga NA
#print(main_df.head())  # Ver cómo quedó el DataFrame

# Crear una nueva columna 'future' que es el valor de cierre futuro desplazado por 'FUTURE_PERIOD_PREDICT'
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

# Crear una columna 'target' que clasifica si el valor futuro es mayor que el valor actual
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)  # Eliminar cualquier fila con NA restante

# Imprimir las primeras filas del DataFrame resultante
#print(main_df.head())


BCH-USD
BTC-USD
ETH-USD
LTC-USD


C:\Users\CRISTIAN CHAVEZ\AppData\Local\Temp\ipykernel_23228\2691810564.py:114: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_df.fillna(method="ffill", inplace=True)  # Si hay huecos en los datos, usar los valores conocidos previamente


In [2]:
# Obtener una lista ordenada de los valores del índice (los tiempos)
times = sorted(main_df.index.values)

# Calcular el punto de corte para el último 5% de los tiempos
last_5pct = sorted(main_df.index.values)[-int(0.05 * len(times))]

# Imprimir el valor del tiempo que representa el inicio del último 5% del conjunto de datos
print(last_5pct)


1534879440


In [3]:
# Crear el DataFrame de validación donde el índice es mayor o igual al punto de corte del último 5%
validation_main_df = main_df[(main_df.index >= last_5pct)]

# Actualizar main_df para que contenga solo los datos hasta antes del último 5%
main_df = main_df[(main_df.index < last_5pct)]

# Preprocesar el DataFrame principal para obtener los datos de entrenamiento
train_x, train_y = preprocess_df(main_df)

# Preprocesar el DataFrame de validación para obtener los datos de validación
validation_x, validation_y = preprocess_df(validation_main_df)

# Imprimir la cantidad de datos de entrenamiento y validación
print(f"train data: {len(train_x)} validation: {len(validation_x)}")

# Imprimir la cantidad de etiquetas 'no compra' (0) y 'compra' (1) en los datos de entrenamiento
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")

# Imprimir la cantidad de etiquetas 'no compra' (0) y 'compra' (1) en los datos de validación
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")


train data: 80460 validation: 4340
Dont buys: 40230, buys: 40230
VALIDATION Dont buys: 2170, buys: 2170
